# 2017-2018

In [2]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]
columns3 =  ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition"]

startTm = datetime.datetime.now()
year = "2018"
df = []
bad = []
readPath = 'J:\\LEVEL2_shenzhen\\' + year + '\\***'

dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[135:]:
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue
        
    # am & pm
    if len(data) == 28:
        path = r'F:\SZ\2018' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        os.system("copy /b am_hq_snap_spot.7z.* am_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_hq_snap_spot.7z')
            bad.append(data + '\\am_hq_snap_spot.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        os.system("copy /b pm_hq_snap_spot.7z.* pm_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_hq_snap_spot.7z')
            bad.append(data + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()

        
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        try:
            assert(am_snap1.shape[1] == len(columns1))
            am_snap1.columns = columns1
        except:
            assert(am_snap1.shape[1] == len(columns3))
            am_snap1.columns = columns3
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        try:
            assert(pm_snap1.shape[1] == len(columns1))
            pm_snap1.columns = columns1
        except:
            assert(pm_snap1.shape[1] == len(columns3))
            pm_snap1.columns = columns3
        snapshot = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data")
   
    # 深交所数据
    else:
        path = r'F:\SZ\2018' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\hq_snap.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\hq_snap.7z')
            bad.append(data + '\\hq_snap.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        
        snapshot = pd.read_table(path1 + "\\hq_snap.txt", header=None, encoding="UTF-8-sig")
        try:
            assert(snapshot.shape[1] == len(columns1))
            snapshot.columns = columns1
        except:
            assert(snapshot.shape[1] == len(columns3))
            snapshot.columns = columns3
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("深交所数据")
        

    print(datetime.datetime.now() - startTm)
    snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 300000)]
    snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)
    s2 = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID").first().reset_index()
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: str(x//1000000000))
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
    s2 = s2.rename(columns={"StockID": "ID", "OrigTime":"date", 'openPrice':"d_open", "PreClosePx":"d_yclose","HighPx":"d_high", "LowPx":"d_low", "close":"d_close", "cum_volume":"d_volume", "cum_amount":"d_amount"})
    s2 = s2[["ID", "date", "d_open", "d_yclose", "d_high", "d_low", "d_close", "d_volume", "d_amount"]]
    
    # 1. check massive missing   
    print(datetime.datetime.now() - startTm)
    date = pd.DataFrame(pd.date_range(start='2019-06-10 08:30:00', end='2019-06-10 18:00:00', freq='s'), columns=["Orig"])
    date["time"] = date["Orig"].apply(lambda x: int(x.strftime("%H%M%S"))*1000)
    date["group"] = date["time"]//30000
    snapshot["group"] = snapshot["time"]//30000
    gl = date[((date["time"] >= 93000000) & (date["time"] < 113000000))|((date["time"] >= 130000000) & (date["time"] <= 145700000))]["group"].unique()
    if len(set(gl) - set(snapshot["group"].unique())) == 0:
        s2["massive_missing"] = 0
    else:
        s2["massive_missing"] = 1
 
    # 2. check each security missing
    print(datetime.datetime.now() - startTm)
    snapshot = snapshot.sort_values(by=["StockID", "OrigTime"])
    dd = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID")["time"].first().reset_index()
    
    data2 = snapshot[(snapshot["time"] > 92500000) & (snapshot["time"] <= 145700000) & (snapshot["cum_volume"] > 0)][["StockID", "cum_volume",
                                                                                                                     "OrigTime", "time"]]
    del snapshot
    data2 = data2.sort_values(by=["StockID", "OrigTime"])
    data2["volume_update"] = data2.groupby("StockID")["cum_volume"].apply(lambda x: x - x.shift(1))
    data2["time1"] = pd.to_datetime(data2["OrigTime"].apply(lambda x: str(x)[:-3]))
    data2["time_interval"] = data2.groupby("StockID")["time1"].apply(lambda x: x - x.shift(1))
    data2["time_interval"] = data2["time_interval"].apply(lambda x: x.seconds)
    data2 = pd.merge(data2, data2[data2["volume_update"] > 0].groupby("StockID")["time_interval"].describe()["75%"].reset_index(), on="StockID")
    data2 = data2.rename(columns={"75%": "75%_time"})
    vu = data2[data2["volume_update"] > 0].groupby("StockID")["volume_update"].apply(lambda x: x.describe([0.9])["90%"]).reset_index()
    vu = vu.rename(columns={"volume_update":"90%"})
    data2 = pd.merge(data2, vu, on="StockID")
    data2["90%"] = data2["90%"].apply(lambda x: max(1000, x))
    data2["indicator"] = 0
    data2.loc[data2[data2["time"] >= 130000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2.loc[data2[data2["time"] >= 93000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2 = data2[data2["indicator"] != 1]    
    if data2[(data2["time_interval"] > 180) & (data2["volume_update"] >= data2["90%"]) & (data2["75%_time"] <= 4)].shape[0] == 0:
        s2["security_missing"] = 0
    else:
        s2["security_missing"] = 1
        
    # 3. check close auction
    print(datetime.datetime.now() - startTm)
    dd["auction"] = np.where(dd["time"]<=145700000, 0, 1)
    dd = dd.rename(columns={"StockID": "ID"})
    s2 = pd.merge(s2, dd[["ID", "auction"]], on="ID")
    
    s2["ID"] = s2["ID"].astype(str)
    s2["ID"] = s2["ID"].apply(lambda x: "SZ" + x.rjust(6, "0"))
    df += [s2]
    print(s2.head())
    print(data + " finished~")
    print(datetime.datetime.now() - startTm)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
df = pd.concat(df).reset_index(drop=True)
display(df)
df.to_pickle(r"F:\\SZ2018_2.pkl")
print(bad)

20180724000000000
am & pm data
0:01:26.945920
0:01:28.665982
0:01:28.998094
0:07:46.536961
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2018-07-24    9.39      9.42    9.59   9.33     9.40  142589416   
1  SZ000002  2018-07-24   24.04     23.75   25.09  24.04    24.38   83281422   
2  SZ000004  2018-07-24   18.36     18.32   18.98  18.05    18.65     531018   
3  SZ000005  2018-07-24    3.02      3.03    3.33   3.02     3.23   43323607   
4  SZ000006  2018-07-24    5.61      5.60    5.86   5.59     5.80   15957013   

       d_amount  massive_missing  security_missing  auction  
0  1.348747e+09                0                 0        1  
1  2.055206e+09                0                 0        1  
2  9.833785e+06                0                 0        1  
3  1.414012e+08                0                 0        1  
4  9.178943e+07                0                 0        1  
J:\LEVEL2_shenzhen\2018\0724 finished~
0:07:46.546934
~~

20180803000000000
am & pm data
1:08:34.630099
1:08:36.259741
1:08:36.560935
1:14:06.313031
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2018-08-03    8.93      8.94    9.10   8.91     8.91  47654662   
1  SZ000002  2018-08-03   21.39     21.38   21.54  21.05    21.08  34607230   
2  SZ000004  2018-08-03   18.50     18.66   19.18  18.26    19.16   1316482   
3  SZ000005  2018-08-03    3.00      2.99    3.03   2.97     2.97   4786371   
4  SZ000006  2018-08-03    5.43      5.45    5.59   5.43     5.49   8661809   

       d_amount  massive_missing  security_missing  auction  
0  4.286415e+08                0                 0        1  
1  7.351437e+08                0                 0        1  
2  2.480696e+07                0                 0        1  
3  1.434166e+07                0                 0        1  
4  4.769966e+07                0                 0        1  
J:\LEVEL2_shenzhen\2018\0803 finished~
1:14:06.323004
~~~~~~~~

20180815000000000
am & pm data
2:11:00.017360
2:11:01.597135
2:11:01.971135
2:16:43.845105
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2018-08-15    9.02      9.00    9.02   8.75     8.79  67044285   
1  SZ000002  2018-08-15   22.39     22.32   22.58  22.22    22.30  47255350   
2  SZ000004  2018-08-15   18.21     18.29   18.38  17.99    18.03    562200   
3  SZ000005  2018-08-15    3.10      3.10    3.10   3.01     3.01   6924900   
4  SZ000006  2018-08-15    5.73      5.69    5.76   5.50     5.51  10677756   

       d_amount  massive_missing  security_missing  auction  
0  5.947595e+08                0                 0        1  
1  1.058912e+09                0                 0        1  
2  1.020215e+07                0                 0        1  
3  2.113374e+07                0                 0        1  
4  5.976134e+07                0                 0        1  
J:\LEVEL2_shenzhen\2018\0815 finished~
2:16:43.855079
~~~~~~~~

20180827000000000
am & pm data
3:12:16.286772
3:12:17.910430
3:12:18.216611
3:18:05.651711
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2018-08-27   10.02     10.03   10.43   9.97    10.43  221842422   
1  SZ000002  2018-08-27   22.98     22.78   24.06  22.85    23.91   59434110   
2  SZ000004  2018-08-27   18.15     18.10   18.32  18.03    18.12     619051   
3  SZ000005  2018-08-27    2.98      2.97    3.05   2.95     3.03    7483761   
4  SZ000006  2018-08-27    5.48      5.51    5.62   5.46     5.60   10289180   

       d_amount  massive_missing  security_missing  auction  
0  2.251528e+09                0                 0        1  
1  1.403514e+09                0                 0        1  
2  1.121888e+07                0                 0        1  
3  2.254888e+07                0                 0        1  
4  5.723786e+07                0                 0        1  
J:\LEVEL2_shenzhen\2018\0827 finished~
3:18:05.660687
~~

20180906000000000
am & pm data
4:10:37.132978
4:10:38.442476
4:10:38.731702
4:16:05.159847
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2018-09-06   10.05     10.05   10.14   9.95     9.97  71177755   
1  SZ000002  2018-09-06   23.32     23.41   23.56  22.69    22.81  37536890   
2  SZ000004  2018-09-06   16.62     17.07   16.97  16.60    16.82    603236   
3  SZ000005  2018-09-06    2.95      2.96    3.00   2.94     2.96   2598301   
4  SZ000006  2018-09-06    5.32      5.30    5.37   5.25     5.26   4738213   

       d_amount  massive_missing  security_missing  auction  
0  7.139757e+08                0                 0        1  
1  8.649946e+08                0                 0        1  
2  1.011195e+07                0                 0        1  
3  7.711378e+06                0                 0        1  
4  2.515184e+07                0                 0        1  
J:\LEVEL2_shenzhen\2018\0906 finished~
4:16:05.169820
~~~~~~~~

...

20181203000000000
am & pm data
11:15:30.669296
11:15:32.392688
11:15:32.717818
11:22:14.377639
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2018-12-03   10.59     10.36   10.66  10.47    10.59  106409717   
1  SZ000002  2018-12-03   26.00     25.44   26.06  24.81    25.73   60814944   
2  SZ000004  2018-12-03   16.39     16.22   16.87  16.39    16.72     413290   
3  SZ000005  2018-12-03    3.03      3.00    3.12   3.03     3.08   11015887   
4  SZ000006  2018-12-03    5.53      5.39    5.68   5.44     5.54   22137903   

       d_amount  massive_missing  security_missing  auction  
0  1.126375e+09                0                 0        1  
1  1.544457e+09                0                 0        1  
2  6.895620e+06                0                 0        1  
3  3.380414e+07                0                 0        1  
4  1.228517e+08                0                 0        1  
J:\LEVEL2_shenzhen\2018\1203 finished~
11:22:14.3876

20181213000000000
am & pm data
12:15:38.757010
12:15:40.268967
12:15:40.576145
12:22:02.046959
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2018-12-13   10.25     10.24   10.48  10.20    10.39  79084521   
1  SZ000002  2018-12-13   26.69     26.35   27.47  26.40    27.10  64446335   
2  SZ000004  2018-12-13   16.59     16.57   17.20  16.41    17.05    705180   
3  SZ000005  2018-12-13    3.00      3.01    3.03   2.99     3.00   6828168   
4  SZ000006  2018-12-13    5.87      5.86    5.92   5.72     5.84  22410360   

       d_amount  massive_missing  security_missing  auction  
0  8.199207e+08                0                 0        1  
1  1.740982e+09                0                 0        1  
2  1.187515e+07                0                 0        1  
3  2.053601e+07                0                 0        1  
4  1.308878e+08                0                 0        1  
J:\LEVEL2_shenzhen\2018\1213 finished~
12:22:02.055935
~~~

20181225000000000
am & pm data
13:13:37.217121
13:13:38.719181
13:13:39.028354
13:19:55.572344
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2018-12-25    9.29      9.42    9.43   9.21     9.34  58661545   
1  SZ000002  2018-12-25   23.42     23.88   24.22  23.38    23.97  42690126   
2  SZ000004  2018-12-25   16.42     16.45   16.42  15.92    16.17    229500   
3  SZ000005  2018-12-25    2.83      2.84    2.83   2.72     2.76   5437752   
4  SZ000006  2018-12-25    5.22      5.33    5.24   5.03     5.17  11680205   

       d_amount  massive_missing  security_missing  auction  
0  5.452356e+08                0                 0        1  
1  1.016226e+09                0                 0        1  
2  3.698461e+06                0                 0        1  
3  1.497640e+07                0                 0        1  
4  5.998073e+07                0                 0        1  
J:\LEVEL2_shenzhen\2018\1225 finished~
13:19:55.582317
~~~

,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount,massive_missing,security_missing,auction
0,SZ000001,2018-07-24,9.39,9.42,9.59,9.33,9.40,142589416,1.348747e+09,0,0,1
1,SZ000002,2018-07-24,24.04,23.75,25.09,24.04,24.38,83281422,2.055206e+09,0,0,1
2,SZ000004,2018-07-24,18.36,18.32,18.98,18.05,18.65,531018,9.833785e+06,0,0,1
3,SZ000005,2018-07-24,3.02,3.03,3.33,3.02,3.23,43323607,1.414012e+08,0,0,1
4,SZ000006,2018-07-24,5.61,5.60,5.86,5.59,5.80,15957013,9.178943e+07,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
228029,SZ300751,2018-12-28,116.80,116.75,120.08,116.25,119.72,934030,1.105244e+08,0,0,1
228030,SZ300752,2018-12-28,59.89,59.85,60.30,56.80,56.99,3798811,2.216577e+08,0,0,1
228031,SZ300753,2018-12-28,40.76,40.29,41.20,39.23,40.12,6627664,2.663134e+08,0,0,1
228032,SZ300756,2018-12-28,64.63,53.86,77.56,64.63,77.56,45144,3.496561e+06,0,0,0


[]


In [4]:
df = pd.concat(df).reset_index(drop=True)
display(df)
df.to_pickle(r"F:\\SZ2018_1.pkl")

,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount,massive_missing,security_missing,auction
0,SZ000001,2018-01-02,13.35,13.30,13.93,13.32,13.70,208159255,2.856544e+09,0,0,1
1,SZ000002,2018-01-02,31.45,31.06,32.99,31.45,32.56,68343350,2.218503e+09,0,0,1
2,SZ000004,2018-01-02,22.29,22.38,22.49,22.00,22.34,626181,1.395100e+07,0,0,1
3,SZ000005,2018-01-02,4.15,4.14,4.50,4.15,4.32,7153934,3.052976e+07,0,0,1
4,SZ000006,2018-01-02,0.00,9.85,0.00,0.00,0.00,0,0.000000e+00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
282558,SZ300743,2018-07-23,69.03,70.33,70.49,66.80,70.45,2453999,1.686896e+08,0,0,1
282559,SZ300745,2018-07-23,67.89,68.25,69.55,64.55,66.55,8210895,5.492921e+08,0,0,1
282560,SZ300746,2018-07-23,33.00,33.23,33.73,32.30,32.59,9094360,2.993082e+08,0,0,1
282561,SZ300747,2018-07-23,207.73,209.80,211.62,203.35,207.97,6270310,1.304048e+09,0,0,1


In [3]:
readPath = r'\\192.168.10.30\Kevin_zhenyu\day_stock_20200416\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs = dataPathLs[[np.array([os.path.basename(i).split('.')[0][:2] == 'SZ' for i in dataPathLs])]]
db = pd.DataFrame()
for p in dataPathLs:
    dayData = pd.read_csv(p, compression='gzip')
    db = pd.concat([db, dayData])
db1 = db[db["date"] == s2["date"].iloc[0]]

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
df = pd.merge(df, db, on=["ID", "date", "d_open", "d_yclose","d_high", "d_low", "d_close", "d_volume"], how="outer")
if sum(df["d_amount_x"].isnull()) != 0:
    diff = df[df["d_amount_x"].isnull()][["ID", "date"]]
print(diff)

            ID        date
2075  SZ001914  2018-01-03


In [6]:
F1 = open("F:\\SZ2018.pkl", 'rb')
d = pickle.load(F1)
d[(d["auction"] == 0) & (d["d_open"] != 0)]
"SZ001914" in d["ID"].values

False

# 2016

In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]
columns3 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "PreClosePx", "openPrice", "HighPx", "LowPx", "close", "NUMTRADES", "cum_volume",
           "cum_amount", "TOTALLONGPOSITION", "PERATIO1", "PERATIO2", "ENDOFDAYMAKER", "TOTALOFFERQTY", "WEIGHTEDAVGOFFERPX", "TOTALBIDQTY", "WEIGHTEDAVGBIDPX"]
columns4 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "ask10p", "ask9p", "ask8p", "ask7p", "ask6p", "ask5p", "ask4p", "ask3p", "ask2p",
           "ask1p", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid6p", "bid7p", "bid8p", "bid9p", "bid10p", "ask10q", "ask9q", "ask8q", "ask7q",
           "ask6q", "ask5q", "ask4q", "ask3q", "ask2q", "ask1q", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "bid6q", "bid7q", "bid8q", "bid9q", 
           "bid10q", "NUMORDERS_B1", "NOORDERS _B1", "ORDERQTY_B1", "NUMORDERS_S1", "NOORDERS _S1", "ORDERQTY_S1"]


year = "2016"
df = []
bad = []
readPath = 'J:\\LEVEL2_shenzhen\\' + year + '\\***\\***'
dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[:3]:
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue
    
    # SZ version A without AM PM, 分卷
    if len(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z.001'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z.001')[0]).split('.')[0].split('_')[2]
        os.mkdir(path + '\\' + d_ate)
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z.001')[0]).split('.')[0]
        os.system("copy /b " + n_ame + ".7z.* " + n_ame + ".7z")
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        a.close()
        
        snapshot = pd.DataFrame()
        if len(np.array(glob.glob(path1 +'\\***'))) == 1:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        for pp in np.array(glob.glob(path1 +'\\***')):
            s1 = pd.read_table(pp, header=None)
            assert(s1.shape[1] == len(columns3))
            s1.columns = columns3
            snapshot = pd.concat([snapshot, s1])
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot = snapshot[snapshot["time"] != 0]
        
        print("SZ version A without AM PM")
        
    # SZ version A AM PM
    # I:\\LEVEL2_shenzhen\\2016\\04\\19\\SZL2_SNAPSHOT_20160419_AM.7z 58
    # I:\\LEVEL2_shenzhen\\2016\\05\\06\\SZL2_SNAPSHOT_20160506_AM.txt 59
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.txt'))) == 1:
        snap1 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns3))
        snap1.columns = columns3
        snap2 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns3))
        snap2.columns = columns3
        snapshot = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot = snapshot[snapshot["time"] != 0]
        
        print("SZ version A AM PM, txt")
                
        
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.7z'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.7z')[0]).split('.')[0].split('_')[2]
        os.mkdir(path + '\\' + d_ate)
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        
        n_ame1 = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***_PM.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame1 + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame1 + '.7z')
            bad.append(data + "\\" + n_ame1 + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        
        snap1 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns3))
        snap1.columns = columns3
        snap2 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns3))
        snap2.columns = columns3
        snapshot = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"] 
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot = snapshot[snapshot["time"] != 0]
        
        print("SZ version A AM PM, .7z")
 
    # am & pm
    # 'I:\\LEVEL2_shenzhen\\2016\\05\\09\\am_snap_level_spot.7z'
    # 'I:\\LEVEL2_shenzhen\\2016\\05\\10\\am_snap_level_spot.7z.001'
    
    elif len(np.array(glob.glob(data +'\\am_hq_snap_spot.7z'))) == 1:
        path = r'F:\SZ\2016' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_hq_snap_spot.7z')
            bad.append(data + '\\am_hq_snap_spot.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        try:
            a = py7zr.SevenZipFile(data + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_hq_snap_spot.7z')
            bad.append(data + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns1))
        am_snap1.columns = columns1       
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns1))
        pm_snap1.columns = columns1
        snapshot = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 未分卷")
        
    elif len(np.array(glob.glob(data +'\\am_snap_level_spot.7z.001'))) == 1:    
        path = r'F:\SZ\2016' 
        os.mkdir(path + '\\' + os.path.basename(data))
        os.chdir(data)
        os.system("copy /b am_hq_snap_spot.7z.* am_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_hq_snap_spot.7z')
            bad.append(data + '\\am_hq_snap_spot.7z')
            continue
        path1 = path + '\\' + os.path.basename(data)
        a.extractall(path = path1)
        a.close()
        
        os.system("copy /b pm_hq_snap_spot.7z.* pm_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_hq_snap_spot.7z')
            bad.append(data + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
      
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns1))
        am_snap1.columns = columns1
        
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns1))
        am_snap1.columns = columns1       
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns1))
        pm_snap1.columns = columns1
        snapshot = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 分卷")
   
  

    snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 300000)]
    snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)
    s2 = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID").first().reset_index()
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: str(x//1000000000))
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
    s2 = s2.rename(columns={"StockID": "ID", "OrigTime":"date", 'openPrice':"d_open", "PreClosePx":"d_yclose","HighPx":"d_high", "LowPx":"d_low", "close":"d_close", "cum_volume":"d_volume", "cum_amount":"d_amount"})
    s2 = s2[["ID", "date", "d_open", "d_yclose", "d_high", "d_low", "d_close", "d_volume", "d_amount"]]
    
    # 1. check massive missing   
    date = pd.DataFrame(pd.date_range(start='2019-06-10 08:30:00', end='2019-06-10 18:00:00', freq='s'), columns=["Orig"])
    date["time"] = date["Orig"].apply(lambda x: int(x.strftime("%H%M%S"))*1000)
    date["group"] = date["time"]//30000
    snapshot["group"] = snapshot["time"]//30000
    gl = date[((date["time"] >= 93000000) & (date["time"] < 113000000))|((date["time"] >= 130000000) & (date["time"] <= 145700000))]["group"].unique()
    if len(set(gl) - set(snapshot["group"].unique())) == 0:
        s2["massive_missing"] = 0
    else:
        s2["massive_missing"] = 1
 
    # 2. check each security missing
    snapshot = snapshot.sort_values(by=["StockID", "OrigTime"])
    dd = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID")["time"].first().reset_index()
    
    data2 = snapshot[(snapshot["time"] > 92500000) & (snapshot["time"] <= 145700000) & (snapshot["cum_volume"] > 0)][["StockID", "cum_volume",
                                                                                                                     "OrigTime", "time"]]
    del snapshot
    data2 = data2.sort_values(by=["StockID", "OrigTime"])    
    data2["volume_update"] = data2.groupby("StockID")["cum_volume"].apply(lambda x: x - x.shift(1))
    data2["time1"] = pd.to_datetime(data2["OrigTime"].apply(lambda x: str(x)[:-3]))
    data2["time_interval"] = data2.groupby("StockID")["time1"].apply(lambda x: x - x.shift(1))
    data2["time_interval"] = data2["time_interval"].apply(lambda x: x.seconds)
    data2 = pd.merge(data2, data2[data2["volume_update"] > 0].groupby("StockID")["time_interval"].describe()["75%"].reset_index(), on="StockID")
    data2 = data2.rename(columns={"75%": "75%_time"})
    vu = data2[data2["volume_update"] > 0].groupby("StockID")["volume_update"].apply(lambda x: x.describe([0.9])["90%"]).reset_index()
    vu = vu.rename(columns={"volume_update":"90%"})
    data2 = pd.merge(data2, vu, on="StockID")
    data2["90%"] = data2["90%"].apply(lambda x: max(1000, x))
    data2["indicator"] = 0
    data2.loc[data2[data2["time"] >= 130000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2.loc[data2[data2["time"] >= 93000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2 = data2[data2["indicator"] != 1]    
    if data2[(data2["time_interval"] > 180) & (data2["volume_update"] >= data2["90%"]) & (data2["75%_time"] <= 4)].shape[0] == 0:
        s2["security_missing"] = 0
    else:
        s2["security_missing"] = 1
        
    # 3. check close auction
    dd["auction"] = np.where(dd["time"]<=145700000, 0, 1)
    dd = dd.rename(columns={"StockID": "ID"})
    s2 = pd.merge(s2, dd[["ID", "auction"]], on="ID")
    
    s2["ID"] = s2["ID"].astype(str)
    s2["ID"] = s2["ID"].apply(lambda x: "SZ" + x.rjust(6, "0"))
    df += [s2]
    print(s2.head())
    print(data + " finished~")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
df = pd.concat(df).reset_index(drop=True)
df.to_pickle(r"F:\\SZ2016.pkl")
print(bad)

Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
F:\SZ\2016\20160104\SZL2_SNAPSHOT_20160104_AM.txt
SZ version A without AM PM
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2016-01-05   11.27     11.33   11.57  11.15    11.40  66326995   
1  SZ000004  2016-01-05   37.50     41.31   40.89  37.18    38.68   3883664   
2  SZ000005  2016-01-05    8.16      9.04    9.22   8.16     8.90  47180448   
3  SZ000006  2016-01-05    9.65     10.38   10.41   9.60    10.04  31166279   
4  SZ000008  2016-01-05    9.69     10.61   11.20   9.69    10.81  22757510   

       d_amount  massive_missing  security_missing  auction  
0  7.555314e+08                0                 1        1  
1  1.505349e+08                0                 1        1  
2  4.149638e+08                0                 1        1  
3  3.111160e+08                0                 1        1  
4  2.418765e+08                0                 1   

In [2]:
readPath = r'\\192.168.10.30\Kevin_zhenyu\day_stock_20200416\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs = dataPathLs[[np.array([os.path.basename(i).split('.')[0][:2] == 'SZ' for i in dataPathLs])]]
db = pd.DataFrame()
for p in dataPathLs:
    dayData = pd.read_csv(p, compression='gzip')
    dayData = dayData[dayData["date"] == "2016-01-05"]
    db = pd.concat([db, dayData])
df1 = pd.merge(df, db, on=["ID", "date", "d_open", "d_yclose","d_high", "d_low", "d_close", "d_volume"], how="outer")
if sum(df1["d_amount_x"].isnull()) != 0:
    diff = df1[df1["d_amount_x"].isnull()][["ID", "date"]]
print(diff)
df[(df["auction"] == 0) & (df["d_open"] != 0)]

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
display(db[db["ID"] == "SZ000022"])
display(db[db["ID"] == "SZ001872"])

,ID,date,d_open,d_high,d_low,d_close,d_yclose,d_cumprodCAA,d_dayReturn,d_5dayReturn,...,allZT,hasZT,isZT,allDT,hasDT,isDT,tmrHalted,haltedDays,marketShares,totalShares
845,SZ000022,2016-01-05,16.83,17.65,16.42,16.98,17.39,0.891702,-0.023577,-0.128784,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464867324.0,644763730.0


,ID,date,d_open,d_high,d_low,d_close,d_yclose,d_cumprodCAA,d_dayReturn,d_5dayReturn,...,allZT,hasZT,isZT,allDT,hasDT,isDT,tmrHalted,haltedDays,marketShares,totalShares
845,SZ001872,2016-01-05,16.83,17.65,16.42,16.98,17.39,0.891702,-0.023577,-0.128784,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464867324.0,644763730.0


In [11]:
db[db.duplicated(keep=False, subset=["date", "d_open", "d_high", "d_low", "d_close", "d_yclose", "d_volume"])]

,ID,date,d_open,d_high,d_low,d_close,d_yclose,d_cumprodCAA,d_dayReturn,d_5dayReturn,...,allZT,hasZT,isZT,allDT,hasDT,isDT,tmrHalted,haltedDays,marketShares,totalShares
965,SZ000043,2016-01-05,10.00,10.89,9.75,10.36,10.83,0.461803,-0.043398,-0.116795,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,666961416.0,666961416.0
845,SZ001872,2016-01-05,16.83,17.65,16.42,16.98,17.39,0.891702,-0.023577,-0.128784,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464867324.0,644763730.0
965,SZ001914,2016-01-05,10.00,10.89,9.75,10.36,10.83,0.461803,-0.043398,-0.116795,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,666961416.0,666961416.0
845,SZ000022,2016-01-05,16.83,17.65,16.42,16.98,17.39,0.891702,-0.023577,-0.128784,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464867324.0,644763730.0
